In [1]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [2]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [3]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [18]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2018-01-01T22:00:00.000000000Z,2968,112.666,112.793,112.582,112.723,112.626,112.786,112.559,112.716,112.706,112.800,112.601,112.730
1,2018-01-02T02:00:00.000000000Z,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,112.728,112.758,112.674,112.694
2,2018-01-02T06:00:00.000000000Z,2226,112.691,112.700,112.249,112.252,112.684,112.694,112.242,112.245,112.698,112.705,112.256,112.258
3,2018-01-02T10:00:00.000000000Z,2056,112.254,112.282,112.086,112.124,112.247,112.275,112.079,112.119,112.260,112.288,112.092,112.130
4,2018-01-02T14:00:00.000000000Z,3781,112.122,112.304,112.058,112.285,112.114,112.297,112.051,112.278,112.129,112.311,112.065,112.292


In [19]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [29]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev
1,2018-01-02T02:00:00.000000000Z,112.720,112.751,112.666,112.688,112.683,112.694,0.085,112.793,112.582,0.211,-1,1
2,2018-01-02T06:00:00.000000000Z,112.691,112.700,112.249,112.252,112.245,112.258,0.451,112.751,112.666,0.085,-1,-1
3,2018-01-02T10:00:00.000000000Z,112.254,112.282,112.086,112.124,112.119,112.130,0.196,112.700,112.249,0.451,-1,-1
4,2018-01-02T14:00:00.000000000Z,112.122,112.304,112.058,112.285,112.278,112.292,0.246,112.282,112.086,0.196,1,-1
5,2018-01-02T18:00:00.000000000Z,112.288,112.354,112.244,112.291,112.262,112.320,0.110,112.304,112.058,0.246,1,1


In [30]:
def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

In [31]:
df = df_raw[['time','mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [33]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL
0,2018-01-02T02:00:00.000000000Z,112.720,112.751,112.666,112.688,112.683,112.694,0.085,112.793,112.582,0.211,-1,1,1
1,2018-01-02T06:00:00.000000000Z,112.691,112.700,112.249,112.252,112.245,112.258,0.451,112.751,112.666,0.085,-1,-1,0
2,2018-01-02T10:00:00.000000000Z,112.254,112.282,112.086,112.124,112.119,112.130,0.196,112.700,112.249,0.451,-1,-1,0
3,2018-01-02T14:00:00.000000000Z,112.122,112.304,112.058,112.285,112.278,112.292,0.246,112.282,112.086,0.196,1,-1,0
4,2018-01-02T18:00:00.000000000Z,112.288,112.354,112.244,112.291,112.262,112.320,0.110,112.304,112.058,0.246,1,1,0


In [34]:
df[df.SIGNAL!=0].shape

(830, 14)

In [35]:
df.shape

(4685, 14)